In [1]:
import pandas as pd
import numpy as np


In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
train_purchases = pd.read_csv("datasets/train_purchases.csv")
train_sessions = pd.read_csv("datasets/train_sessions.csv")
item_features = pd.read_csv("datasets/item_features.csv")
candidate_items = pd.read_csv("datasets/candidate_items.csv")

In [12]:
test_leaderboard = pd.read_csv("datasets/test_leaderboard_sessions.csv")

In [13]:
test_leaderboard.head()

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692


In [3]:
train_purchases.head()

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


In [4]:
train_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [9]:
print("Category number : ",len(item_features["feature_category_id"].unique()))

Category number :  73


In [12]:
item_features.head()

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75


In [6]:
candidate_items.head()

,item_id
0,4
1,8
2,9
3,19
4,20


In [14]:
train_purchases[train_purchases["item_id"] == 15085]

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
81109,362500,15085,2020-11-20 15:43:18.18
156247,695904,15085,2020-12-11 08:57:55.779
208983,931085,15085,2020-12-09 14:30:26.118
295572,1315209,15085,2020-12-17 21:23:08.308
302413,1344939,15085,2021-01-18 18:14:13.989
310688,1381862,15085,2020-12-11 22:39:54.123
350089,1556254,15085,2020-12-06 21:38:37.59
363762,1616554,15085,2020-11-29 12:24:28.595
395474,1757080,15085,2020-12-11 12:46:46.343


In [62]:
fulldf = train_purchases[train_purchases.index <= 1000].copy()

In [63]:
def get_features():
    column = []
    for session in fulldf["session_id"]:
        history = train_sessions[train_sessions["session_id"] == session]["item_id"]
        res = ""
        for k,itm in enumerate(history):
            item = (item_features[item_features["item_id"] == itm]["feature_category_id"].astype("str") 
                    + item_features[item_features["item_id"] == itm]["feature_value_id"].astype("str")).astype("str").str.cat(sep=" ")

            if k == 0:
                res += item
            else:
                res += " " + item
                        
        column.append(res)
    return column

In [64]:
fulldf["features"] = get_features() 

In [67]:
cm = CountVectorizer().fit_transform(fulldf["features"])

In [68]:
cs = cosine_similarity(cm)
print(cs)

[[1.         0.05270463 0.14133305 ... 0.05407381 0.15891043 0.11785113]
 [0.05270463 1.         0.28492065 ... 0.10259784 0.39196475 0.372678  ]
 [0.14133305 0.28492065 1.         ... 0.22354067 0.51544084 0.38725804]
 ...
 [0.05407381 0.10259784 0.22354067 ... 1.         0.18560647 0.24853371]
 [0.15891043 0.39196475 0.51544084 ... 0.18560647 1.         0.59554321]
 [0.11785113 0.372678   0.38725804 ... 0.24853371 0.59554321 1.        ]]


In [87]:
def get_real_id_for_item(itm_id):
    return fulldf[fulldf["item_id"] == itm_id].values[0][0]

real_id = get_real_id_for_item(15085)

,session_id,item_id,date,features
401,1943,17186,2020-08-29 11:41:51.413,56365 72557 17378 19545 41759 3793 32902 46825...


In [89]:
scores = list(enumerate(cs[real_id]))

In [90]:
sorted_scores = sorted(scores, key = lambda x: x[1], reverse=True)
sorted_scores = sorted_scores[1:]

In [91]:
print("TOP 5 >> Recommended products are : ")
for rec in sorted_scores[0:5]:
    print(fulldf[fulldf.index == rec][item_id])

[(401, 0.9107824399646117),
 (637, 0.8965421872576498),
 (416, 0.8927250790585792),
 (197, 0.8884524039059751),
 (79, 0.8862610304466845)]